# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 31 2023 1:55PM,256318,10,PRF-50915,Bio-PRF,Released
1,Jan 31 2023 1:55PM,256318,10,PRF-50985,Bio-PRF,Released
2,Jan 31 2023 1:55PM,256318,10,PRF-50991,Bio-PRF,Released
3,Jan 31 2023 1:55PM,256318,10,PRF-50998,Bio-PRF,Released
4,Jan 31 2023 1:55PM,256318,10,PRF-51004,Bio-PRF,Released
5,Jan 31 2023 1:41PM,256317,21,639882,"NBTY Global, Inc.",Released
6,Jan 31 2023 1:36PM,256316,21,640015,"NBTY Global, Inc.",Released
7,Jan 31 2023 1:36PM,256305,21,639829,"NBTY Global, Inc.",Released
8,Jan 31 2023 1:21PM,256313,10,Yusen-9099485,Yusen – 3D Matrix,Released
9,Jan 31 2023 1:14PM,256311,10,0086342341,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,256312,Released,1
40,256313,Released,1
41,256316,Released,1
42,256317,Released,1
43,256318,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256312,NaN,NaN,1.0
256313,NaN,NaN,1.0
256316,NaN,NaN,1.0
256317,NaN,NaN,1.0
256318,NaN,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256240,25.0,0.0,12.0
256245,0.0,0.0,1.0
256252,0.0,3.0,3.0
256259,3.0,6.0,3.0
256260,0.0,2.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256240,25,0,12
256245,0,0,1
256252,0,3,3
256259,3,6,3
256260,0,2,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256240,25,0,12
1,256245,0,0,1
2,256252,0,3,3
3,256259,3,6,3
4,256260,0,2,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256240,25,,12
1,256245,,,1
2,256252,,3,3
3,256259,3,6,3
4,256260,,2,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 31 2023 1:55PM,256318,10,Bio-PRF
5,Jan 31 2023 1:41PM,256317,21,"NBTY Global, Inc."
6,Jan 31 2023 1:36PM,256316,21,"NBTY Global, Inc."
7,Jan 31 2023 1:36PM,256305,21,"NBTY Global, Inc."
8,Jan 31 2023 1:21PM,256313,10,Yusen – 3D Matrix
9,Jan 31 2023 1:14PM,256311,10,ISDIN Corporation
30,Jan 31 2023 1:13PM,256312,10,ISDIN Corporation
31,Jan 31 2023 1:11PM,256309,10,"Methapharm, Inc."
32,Jan 31 2023 1:08PM,256308,16,Sartorius Bioprocess Solutions
33,Jan 31 2023 12:46PM,256302,19,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 31 2023 1:55PM,256318,10,Bio-PRF,,,5
1,Jan 31 2023 1:41PM,256317,21,"NBTY Global, Inc.",,,1
2,Jan 31 2023 1:36PM,256316,21,"NBTY Global, Inc.",,,1
3,Jan 31 2023 1:36PM,256305,21,"NBTY Global, Inc.",,,1
4,Jan 31 2023 1:21PM,256313,10,Yusen – 3D Matrix,,,1
5,Jan 31 2023 1:14PM,256311,10,ISDIN Corporation,,,21
6,Jan 31 2023 1:13PM,256312,10,ISDIN Corporation,,,1
7,Jan 31 2023 1:11PM,256309,10,"Methapharm, Inc.",,,1
8,Jan 31 2023 1:08PM,256308,16,Sartorius Bioprocess Solutions,,,1
9,Jan 31 2023 12:46PM,256302,19,"Emersa Waterbox, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 31 2023 1:55PM,256318,10,Bio-PRF,5,,
1,Jan 31 2023 1:41PM,256317,21,"NBTY Global, Inc.",1,,
2,Jan 31 2023 1:36PM,256316,21,"NBTY Global, Inc.",1,,
3,Jan 31 2023 1:36PM,256305,21,"NBTY Global, Inc.",1,,
4,Jan 31 2023 1:21PM,256313,10,Yusen – 3D Matrix,1,,
5,Jan 31 2023 1:14PM,256311,10,ISDIN Corporation,21,,
6,Jan 31 2023 1:13PM,256312,10,ISDIN Corporation,1,,
7,Jan 31 2023 1:11PM,256309,10,"Methapharm, Inc.",1,,
8,Jan 31 2023 1:08PM,256308,16,Sartorius Bioprocess Solutions,1,,
9,Jan 31 2023 12:46PM,256302,19,"Emersa Waterbox, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 31 2023 1:55PM,256318,10,Bio-PRF,5,,
1,Jan 31 2023 1:41PM,256317,21,"NBTY Global, Inc.",1,,
2,Jan 31 2023 1:36PM,256316,21,"NBTY Global, Inc.",1,,
3,Jan 31 2023 1:36PM,256305,21,"NBTY Global, Inc.",1,,
4,Jan 31 2023 1:21PM,256313,10,Yusen – 3D Matrix,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 31 2023 1:55PM,256318,10,Bio-PRF,5.0,NaN,NaN
1,Jan 31 2023 1:41PM,256317,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jan 31 2023 1:36PM,256316,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jan 31 2023 1:36PM,256305,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Jan 31 2023 1:21PM,256313,10,Yusen – 3D Matrix,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 31 2023 1:55PM,256318,10,Bio-PRF,5.0,0.0,0.0
1,Jan 31 2023 1:41PM,256317,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jan 31 2023 1:36PM,256316,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jan 31 2023 1:36PM,256305,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Jan 31 2023 1:21PM,256313,10,Yusen – 3D Matrix,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5125805,93.0,10.0,3.0
15,768776,25.0,87.0,28.0
16,768894,2.0,1.0,0.0
18,256276,1.0,0.0,0.0
19,1281384,7.0,3.0,0.0
20,256300,1.0,0.0,0.0
21,768938,3.0,0.0,0.0
22,256245,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5125805,93.0,10.0,3.0
1,15,768776,25.0,87.0,28.0
2,16,768894,2.0,1.0,0.0
3,18,256276,1.0,0.0,0.0
4,19,1281384,7.0,3.0,0.0
5,20,256300,1.0,0.0,0.0
6,21,768938,3.0,0.0,0.0
7,22,256245,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,93.0,10.0,3.0
1,15,25.0,87.0,28.0
2,16,2.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,7.0,3.0,0.0
5,20,1.0,0.0,0.0
6,21,3.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,93.0
1,15,Released,25.0
2,16,Released,2.0
3,18,Released,1.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,3.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,10.0,87.0,1.0,0.0,3.0,0.0,0.0,0.0
Released,93.0,25.0,2.0,1.0,7.0,1.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,3.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,87.0,1.0,0.0,3.0,0.0,0.0,0.0
2,Released,93.0,25.0,2.0,1.0,7.0,1.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,3.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,87.0,1.0,0.0,3.0,0.0,0.0,0.0
2,Released,93.0,25.0,2.0,1.0,7.0,1.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()